# 02 – Binary Modelltraining (Diabetes-Risiko: Ja/Nein)

**Ziel:** Wir trainieren ein binäres Modell, das nur zwischen **kein Diabetes** (0) und **Diabetes-Risiko** (1) unterscheidet.
**Warum?** Die Klasse „Prädiabetes“ ist im Datensatz extrem selten und schwer abzugrenzen, daher ist ein binäres Setting oft stabiler.

## 1) Datenbasis

Wir laden den bereits feature-engineerten **Binary-Datensatz** (`diabetes_fe_binary.csv`), damit Training und App später exakt dieselben Features verwenden.

In [1]:
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score
import joblib

PROJECT_ROOT = Path("..")
DATA_PATH = PROJECT_ROOT / "data" / "processed" / "diabetes_fe_binary.csv"
MODEL_OUT = PROJECT_ROOT / "models" / "diabetes_binary_model.joblib"

df = pd.read_csv(DATA_PATH)
df.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,Education,Income,inactive,cardio_risk_sum,low_fruits,low_veggies,lifestyle_risk_sum,poor_health,mental_physical_burden,Diabetes_binary
0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,...,4.0,3.0,1,2.0,1,0,3.0,1,33.0,0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,...,6.0,1.0,0,0.0,1,1,3.0,0,0.0,0
2,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,...,4.0,8.0,1,2.0,0,1,2.0,1,60.0,0
3,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,...,3.0,6.0,0,1.0,0,0,0.0,0,0.0,0
4,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,...,5.0,4.0,0,2.0,0,0,0.0,0,3.0,0


## 2) Train/Test Split

Wir splitten die Daten **stratifiziert**, damit das Verhältnis von 0/1 im Train- und Testset gleich bleibt und die Evaluation fair ist.

In [2]:
target = "Diabetes_binary"
X = df.drop(columns=[target])
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)
print("Train label distribution:\n", y_train.value_counts(normalize=True).round(3))

Train shape: (202944, 28)
Test shape: (50736, 28)
Train label distribution:
 Diabetes_binary
0    0.842
1    0.158
Name: proportion, dtype: float64


## 3) Modell

Wir nutzen **Logistic Regression** als robuste, interpretierbare Baseline und setzen `class_weight="balanced"`, um das (immer noch vorhandene) Ungleichgewicht zu berücksichtigen.

In [3]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        max_iter=2000,
        class_weight="balanced"
    ))
])

pipe.fit(X_train, y_train)
print("✅ Modell trainiert")

✅ Modell trainiert


## 4) Evaluation

Wir betrachten neben dem Report auch die **Confusion Matrix**, weil sie zeigt, ob das Modell Risiko-Fälle übersieht (False Negatives).
Zusätzlich nutzen wir **F1 (für Klasse 1)** und **ROC-AUC**, um die Qualität bei ungleichen Klassen besser zu bewerten als nur Accuracy.

In [4]:
y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1]

print("=== Classification Report ===")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

print(f"F1 (positive class): {f1:.4f}")
print(f"ROC-AUC: {auc:.4f}")

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.94      0.72      0.82     42741
           1       0.34      0.76      0.47      7995

    accuracy                           0.73     50736
   macro avg       0.64      0.74      0.64     50736
weighted avg       0.85      0.73      0.76     50736

Confusion Matrix:
[[30887 11854]
 [ 1895  6100]]
F1 (positive class): 0.4702
ROC-AUC: 0.8175


## 5) Speichern

Wir speichern das trainierte Pipeline-Modell als `.joblib`, damit die Web-App es direkt laden und reproduzierbar Vorhersagen machen kann.

In [5]:
joblib.dump(pipe, MODEL_OUT)
print("✅ Modell gespeichert:", MODEL_OUT)

✅ Modell gespeichert: ..\models\diabetes_binary_model.joblib
